In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd '/content/drive/My Drive/nlp'

Mounted at /content/drive
/content/drive/My Drive/nlp


In [ ]:
import string
import numpy as np
from operator import itemgetter

LEN_UNI = 0


In [ ]:
LEN_BI = 0

In [ ]:
def create_corpus():
  corpus = {}
  global LEN_UNI
  with open('/content/drive/My Drive/nlp/polish_corpora.txt', 'r', encoding = 'utf8') as file:
   for i, line in enumerate(file):
    words = line.split()
    for word in words:
      if word in corpus:
        num = corpus[word]
        corpus[word] = num + 1
        LEN_UNI+=1
      else:
        corpus[word] = 1
        LEN_UNI+=1
  return corpus

corpus = create_corpus()

In [ ]:
print(corpus['koń'])
print(LEN_UNI)

1614
451846640


In [ ]:
def create_bigrams():
  bigrams_dict = {}
  global LEN_BI
  with open('/content/drive/My Drive/nlp/poleval_2grams.txt', 'r', encoding = 'utf8') as file:
    for i, line in enumerate(file):
      
      words = line.split()
      how_many, word1, word2 = int(words[0]), words[1], words[2]  #cast to int
      #print(words)
      if how_many < 5:
        continue
      else:
        if word1 in bigrams_dict:
          LEN_BI+=how_many
          bigrams_dict[word1].append((how_many, word2))
        else:
          bigrams_dict[word1] = [(how_many, word2)]
          LEN_BI+=how_many
        #if word2 in bigrams_dict:
        #  bigrams_dict[word2].append((how_many, word1))
        #else:
        #  bigrams_dict[word2] = [(how_many, word1)]

  return bigrams_dict
bigrams_dict = create_bigrams()
print(LEN_BI)

440151062


In [ ]:
len(bigrams_dict)

420755

In [ ]:
def ppmi(word, unigrams, bigrams, k=10):
  results = []
  #len_uni = len(unigrams)
  #len_bi = len(bigrams)
  res_words = []
  res_num = []

  for elements in bigrams[word]:
    if elements[1] == '<BOS>': continue
    if elements[1] == '<EOS>': continue
    if elements[1] not in unigrams:
      unigrams[elements[1]] = 1
    if unigrams[elements[1]] == 0:
      continue
    log = np.log((elements[0]) / (unigrams[word] * unigrams[elements[1]])*LEN_UNI**2 /LEN_BI)
    if log <= 0:
      log = 0
    if log>0:
      res_words.append(elements[1])
      res_num.append(log)
      #results.append((elements[1], log))
  #print()
  res_num = (res_num-min(res_num))/(max(res_num)-min(res_num))
  for i in range(len(res_num)):
    results.append((res_words[i], res_num[i]))
  results = sorted(results, key=itemgetter(1), reverse=True)
  return results[:k]

res1 = ppmi('kobieta', corpus, bigrams_dict)


In [ ]:
res1

[('sponsorka', 1.0),
 ('65-letnia', 0.9148588270496669),
 ('spodziewająca', 0.8437810999569512),
 ('ciężarna', 0.8206136893372333),
 ('zasponsoruje', 0.8148251845973905),
 ('karmiąca', 0.7940810652056924),
 ('czytająca', 0.7737650500199527),
 ('poddająca', 0.7412604628451471),
 ('podająca', 0.7059435808185901),
 ('zarabiająca', 0.700514980729876)]

In [ ]:
def poisson_stirling_approx(word, unigrams, bigrams, k=10):
  results = []
  res_words = []
  res_num = []
  len_uni = LEN_UNI#len(unigrams)
  len_bi = LEN_BI#len(bigrams)
  for elements in bigrams[word]:
    if elements[1] == '<BOS>': continue
    if elements[1] == '<EOS>': continue
    if elements[1] not in unigrams:
      unigrams[elements[1]] = 1
    if unigrams[elements[1]]==0:
      continue
    psm = elements[0]*(np.log(elements[0]) - np.log(unigrams[word]*unigrams[elements[1]]/len_uni) - 1)
    res_words.append(elements[1])
    res_num.append(psm)
    #results.append((elements[1], psm))

#feature scaling

  res_num = (res_num-min(res_num))/(max(res_num)-min(res_num))
  for i in range(len(res_num)):
    results.append((res_words[i], res_num[i]))  
  results = sorted(results, key=itemgetter(1), reverse=True)
  return results[:k]

res1 = poisson_stirling_approx('kobieta', corpus, bigrams_dict)


In [ ]:
def ppmi_poisson_mix(word, unigrams, bigrams, k=10):
  results = []
  len_uni = LEN_UNI#len(unigrams)
  len_bi = LEN_BI#len(bigrams)
  res_words = []
  res_num = []
  res_num2 = []
  for elements in bigrams[word]:
    if elements[1] == '<BOS>': continue
    if elements[1] == '<EOS>': continue
    if elements[1] not in unigrams:
      unigrams[elements[1]] = 1
    if unigrams[elements[1]] == 0:
      continue
    log = np.log((elements[0]) / (unigrams[word] * unigrams[elements[1]])*len_uni**2 /len_bi)
    psm = elements[0]*(np.log(elements[0]) - np.log(unigrams[word]*unigrams[elements[1]]/len_uni) - 1)
    if log <= 0:
      log = 0
    if log>0:
      res_words.append(elements[1])
      res_num.append(log)
      res_num2.append(psm)
      #results.append((elements[1], log))

  res_num2 = (res_num2-min(res_num2))/(max(res_num2)-min(res_num2))
  res_num = (res_num-min(res_num))/(max(res_num)-min(res_num))
  
  res_num = [res_num[i] * res_num2[i] for i in range(len(res_num))]
  for i in range(len(res_num)):
    results.append((res_words[i], res_num[i]))
  results = sorted(results, key=itemgetter(1), reverse=True)
  return results[:k]


ppmi_poisson_mix('kobieta', corpus, bigrams_dict)

[('ciężarna', 0.633380719717717),
 ('zasponsoruje', 0.5291180323900957),
 ('sponsorka', 0.5176077250216335),
 ('65-letnia', 0.5034039838799583),
 ('spodziewająca', 0.41280427101344236),
 ('kierująca', 0.40823893399143124),
 ('karmiąca', 0.4075393661520379),
 ('szuka', 0.3953411833687385),
 ('powinna', 0.39119290344293806),
 ('siedząca', 0.3742776135436099)]

## porównanie ppmi i poisson striling dla zestawu słów

In [ ]:
words_to_test = ['kobieta', 'mężczyzna', 'szkoła', 'okoń', 'zegarek', 'zamek', 'śruba', 'dysk', 'czajnik', 'samochód']

In [ ]:
def words_testing_ppmi_poisson(words):
  for word in words:
    res_ppmi = ppmi(word, corpus, bigrams_dict)
    res_psa = poisson_stirling_approx(word, corpus, bigrams_dict)
    print(word)
    print('{:<3}{:<30s}{:<6}{:<22s}{:<6}'.format('id', 'ppmi', '', 'psa',''))
    for i in range(len(res_ppmi)):
      print('{:<3}{:<30s}{:<6.3f}{:<25s}{:6.3f}'.format(i,res_ppmi[i][0],res_ppmi[i][1], res_psa[i][0],res_psa[i][1]) )
    print('')

words_testing_ppmi_poisson(words_to_test)

kobieta
id ppmi                                psa                         
0  sponsorka                     1.000 ma                        1.000
1  65-letnia                     0.915 powinna                   0.942
2  spodziewająca                 0.844 nie                       0.821
3  ciężarna                      0.821 może                      0.814
4  zasponsoruje                  0.815 ciężarna                  0.772
5  karmiąca                      0.794 szuka                     0.734
6  czytająca                     0.774 ,                         0.715
7  poddająca                     0.741 jest                      0.704
8  podająca                      0.706 była                      0.696
9  zarabiająca                   0.701 chce                      0.687

mężczyzna
id ppmi                                psa                         
0  podający                      1.000 został                    1.000
1  włamał                        0.939 żonaty                   

## połaczenie strirling i ppmi

In [ ]:
def words_testing_ppmi_poisson_with_mix(words):
  for word in words:
    res_ppmi = ppmi(word, corpus, bigrams_dict)
    res_psa = poisson_stirling_approx(word, corpus, bigrams_dict)
    res_mix = ppmi_poisson_mix(word, corpus, bigrams_dict)

    print(word)
    print('{:<3}{:<30s}{:<6}{:<22s}{:<6}{:<25s}{:<6}'.format('id', 'ppmi', '', 'psa', '', 'psa+ppmi',''))
    for i in range(len(res_ppmi)):
      print('{:<3}{:<30s}{:<6.3f}{:<22s}{:<6.3f}{:<25s}{:6.3f}'.format(i,res_ppmi[i][0],res_ppmi[i][1], res_psa[i][0],res_psa[i][1],res_mix[i][0],res_mix[i][1]) )
    print('')
words_testing_ppmi_poisson_with_mix(words_to_test)

kobieta
id ppmi                                psa                         psa+ppmi                       
0  sponsorka                     1.000 ma                    1.000 ciężarna                  0.633
1  65-letnia                     0.915 powinna               0.942 zasponsoruje              0.529
2  spodziewająca                 0.844 nie                   0.821 sponsorka                 0.518
3  ciężarna                      0.821 może                  0.814 65-letnia                 0.503
4  zasponsoruje                  0.815 ciężarna              0.772 spodziewająca             0.413
5  karmiąca                      0.794 szuka                 0.734 kierująca                 0.408
6  czytająca                     0.774 ,                     0.715 karmiąca                  0.408
7  poddająca                     0.741 jest                  0.704 szuka                     0.395
8  podająca                      0.706 była                  0.696 powinna                   0.391
9 

In [ ]:
file1 = open('supertags.txt')
tags = {}
for i in file1:
  words = i.split()
  tags[words[0]] = words[1]

In [ ]:
#create sth to hold tags number of occurences
def create_tags_unigrams(corpus, tags_corp): 
  tags_unigrams = {}
  for word in corpus:
    if word in tags_corp:
      tag = tags_corp[word]
      if tag in tags_unigrams:
        tags_unigrams[tag].append((word, corpus[word]))
      else:
        tags_unigrams[tag] = [(word, corpus[word])]
    else:
      new_tag = ('^' + word)[-3:]
      tags_unigrams[new_tag] = [(word, corpus[word])]
  return tags_unigrams
  
tags_unigrams = create_tags_unigrams(corpus, tags)

In [ ]:
k=0
for i in tags_unigrams:
  print(i, tags_unigrams[i])
  k+=1
  if k==10:
    break

tem [('Gruppensystem', 1)]
aty [('wybór-krawaty', 1)]
esz [('tata-będziesz', 1)]
T1085 [('kupować', 4089), ('stawiać', 3055), ('odbywać', 9593), ('zdawać', 2892), ('analizować', 1688), ('przyjmować', 4002), ('szukać', 10394), ('podejmować', 8134), ('chwalić', 898), ('pomijać', 552), ('świadczyć', 4432), ('wliczać', 205), ('zwiększać', 2474), ('wykorzystywać', 4693), ('hamować', 549), ('głosić', 597), ('mówić', 29480), ('pamiętać', 22926), ('upatrywać', 686), ('otrzymywać', 3724), ('wprowadzać', 5826), ('odkrywać', 557), ('poznawać', 877), ('dostarczać', 1451), ('wykonywać', 10119), ('doceniać', 432), ('rozwijać', 8318), ('likwidować', 1453), ('utrudniać', 1142), ('unikać', 3143), ('rozumieć', 5626), ('traktować', 8050), ('kontrolować', 4243), ('wchodzić', 3872), ('prosić', 6518), ('rozwiązywać', 2240), ('nadawać', 1179), ('akceptować', 689), ('powtarzać', 3106), ('dyskutować', 5940), ('tworzyć', 11305), ('badać', 1815), ('pełnić', 5821), ('utrzymywać', 3459), ('chronić', 7079), ('infor

In [ ]:
def last_letters_tag(word, num):
  return ('^' + word)[-3:]


In [ ]:
def create_tags_bigrams(bigrams, tags):
  tags_bigrams={}
  for word1 in bigrams: 
    for word2 in bigrams[word1]:
      if (word1 in tags) and (word2[1] in tags):
        tag1, tag2 = tags[word1], tags[word2[1]]
        if (tag1, tag2) in tags_bigrams:
          tags_bigrams[(tag1, tag2)].append((word1, word2[1], word2[0]))
        else:
          tags_bigrams[(tag1, tag2)] = [(word1, word2[1], word2[0])]

      elif word1 in tags:
        tag1 = tags[word1]
        tag2 =  last_letters_tag(word2[1], 3)
        if (tag1, tag2) in tags_bigrams:
          tags_bigrams[(tag1, tag2)].append((word1, word2[1], word2[0]))
        else:
          tags_bigrams[(tag1, tag2)] = [(word1, word2[1], word2[0])]

      elif word2[1] in tags:
        tag1 = last_letters_tag(word1, 3)
        tag2 =  tags[word2[1]]
        if (tag1, tag2) in tags_bigrams:
          tags_bigrams[(tag1, tag2)].append((word1, word2[1], word2[0]))
        else:
          tags_bigrams[(tag1, tag2)] = [(word1, word2[1], word2[0])]

      else:
        tag1 = last_letters_tag(word1, 3)
        tag2 = last_letters_tag(word2[1], 3)
        if (tag1, tag2) in tags_bigrams:
          tags_bigrams[(tag1, tag2)].append((word1, word2[1], word2[0]))
        else:
          tags_bigrams[(tag1, tag2)] = [(word1, word2[1], word2[0])]


  return tags_bigrams

tags_bigrams = create_tags_bigrams(bigrams_dict, tags)

In [ ]:
k=0
for i in tags_bigrams:
  print(i, tags_bigrams[i])
  k+=1
  if k==10:
    break

('T3798', 'T3046') [('rozdrobniona', 'sieć', 11), ('uruchomiona', 'sprzedaż', 8), ('uruchomiona', 'pomoc', 10), ('prowadzona', 'sprzedaż', 30), ('prowadzona', 'działalność', 285), ('prowadzona', 'księgowość', 8), ('ustalana', 'wielkość', 5), ('ustalana', 'wartość', 5), ('ustalana', 'wysokość', 15), ('zawieszona', 'działalność', 17), ('otwarta', 'dłoń', 7), ('otwarta', 'część', 7), ('otwarta', 'możliwość', 6), ('otwarta', 'przestrzeń', 114), ('przyznana', 'możliwość', 6), ('przyznana', 'nieruchomość', 14), ('przyznana', 'pomoc', 65), ('przyznana', 'płatność', 11), ('zachowana', 'ciągłość', 32), ('zachowana', 'możliwość', 12), ('zachowana', 'równość', 7), ('zachowana', 'płynność', 10), ('zachowana', 'kolejność', 8), ('zachowana', 'cerkiew', 5), ('zachowana', 'część', 31), ('zachowana', 'pieczęć', 17), ('znana', 'skamieniałość', 10), ('znana', 'opowieść', 5), ('znana', 'ilość', 6), ('znana', 'osobistość', 11), ('znana', 'treść', 11), ('znana', 'wartość', 12), ('znana', 'twarz', 5), ('znan

## ppmi + tag

In [ ]:
def ppmi_word_tag(word, unigrams, bigrams, tags, unitags, bitags, k=20):


  results = []
  len_uni = LEN_UNI#len(unigrams)
  len_bi = LEN_BI#len(bigrams)
  res_words = []
  res_num = []
  res_num2 = []

  for elements in bigrams[word]:
    if elements[1] == '<BOS>': continue
    if elements[1] == '<EOS>': continue
    if elements[1] not in unigrams:
      unigrams[elements[1]] = 1
    if unigrams[elements[1]] == 0:
      continue
    log = np.log((elements[0]) / (unigrams[word] * unigrams[elements[1]])*len_uni**2 /len_bi)
    
    if word in tags:
      tag1 = tags[word]
    else:
      tag1 = last_letters_tag(word,3)
    if elements[1] in tags:
      tag2 = tags[elements[1]]
    else:
      tag2 = last_letters_tag(elements[1],3)

    bitag_sum = 0
    unitag1_sum = 0
    unitag2_sum = 0
    #print(tag1, tag2)
    for tag_pair in bitags[(tag1,tag2)]:
      bitag_sum+=tag_pair[2]
    for tag_w in unitags[tag1]:
      unitag1_sum+=tag_w[1]
    for tag_w in unitags[tag2]:
      unitag2_sum+=tag_w[1]
   
    log_tag = np.log((bitag_sum / (unitag1_sum * unitag2_sum)*len_uni**2 /len_bi))

    if log <= 0:
      log = 0
    if log_tag<0:
      log_tag = 0
    if log>0 and log_tag>0:
      res_words.append(elements[1])
      res_num.append(log)
      res_num2.append(log_tag)
      #results.append((elements[1], log))
  #print()
  res_num = (res_num-min(res_num))/(max(res_num)-min(res_num))
  res_num2 = (res_num2-min(res_num2))/(max(res_num2)-min(res_num2))
  for i in range(len(res_num)):
    results.append((res_words[i], res_num[i]*res_num2[i]))
  results = sorted(results, key=itemgetter(1), reverse=True)
  return results[:k]

ppmi_word_tag('kobieta', corpus, bigrams_dict, tags, tags_unigrams, tags_bigrams)




[('65-letnia', 0.6336375314558811),
 ('ktora', 0.2708677405240569),
 ('płacze', 0.26472785725474174),
 ('czuje', 0.24205899926785523),
 ('zniknęła', 0.22857714050212816),
 ('może', 0.22277041439629275),
 ('nagle', 0.19516340509666805),
 ('ciężarna', 0.18569463393504185),
 ('spodziewająca', 0.16397704913723868),
 ('karmiąca', 0.15431854287187505),
 ('tuż', 0.15246475296819303),
 ('czytająca', 0.15037040961722536),
 ('wkrótce', 0.1502160746473171),
 ('nigdy', 0.1459926089453335),
 ('źle', 0.14436300585361597),
 ('poddająca', 0.14405359796000683),
 ('cmentarna', 0.14039317695239698),
 ('podająca', 0.1371902561528269),
 ('zarabiająca', 0.13613528369191386),
 ('kierująca', 0.13550953963920087)]

In [ ]:
def words_testing_all_methods(words):
  
  for word in words:
    res_ppmi = ppmi(word, corpus, bigrams_dict)
    res_psa = poisson_stirling_approx(word, corpus, bigrams_dict)
    res_mix = ppmi_poisson_mix(word, corpus, bigrams_dict)
    res_tag = ppmi_word_tag(word, corpus, bigrams_dict, tags, tags_unigrams, tags_bigrams)
    print(word)
    print('{:<3}{:<30s}{:<6}{:<20s}{:<6}{:<25s}{:<6}{:<20}{:<6}'.format('id', 'ppmi', '', 'psa', '', 'psa+ppmi','','tags',''))
    for i in range(len(res_ppmi)):
      print('{:<3}{:<30s}{:<6.2f}{:<20s}{:<6.2f}{:<25s}{:<6.2f}{:<20}{:<6.2f}'.format(i,res_ppmi[i][0],res_ppmi[i][1], \
                            res_psa[i][0],res_psa[i][1],res_mix[i][0],res_mix[i][1],res_tag[i][0],res_tag[i][1]) )
    print('')
words_testing_all_methods(words_to_test)

kobieta
id ppmi                                psa                       psa+ppmi                       tags                      
0  sponsorka                     1.00  ma                  1.00  ciężarna                 0.63  65-letnia           0.63  
1  65-letnia                     0.91  powinna             0.94  zasponsoruje             0.53  ktora               0.27  
2  spodziewająca                 0.84  nie                 0.82  sponsorka                0.52  płacze              0.26  
3  ciężarna                      0.82  może                0.81  65-letnia                0.50  czuje               0.24  
4  zasponsoruje                  0.81  ciężarna            0.77  spodziewająca            0.41  zniknęła            0.23  
5  karmiąca                      0.79  szuka               0.73  kierująca                0.41  może                0.22  
6  czytająca                     0.77  ,                   0.72  karmiąca                 0.41  nagle               0.20  
7  podda

In [ ]:
def chem(n_ag, n_k1, n_k2, n_aqua, n_k3):
  m_ag = 107.8682
  m_c = 12.011
  m_h = 1.008
  m_o = 15.999
  m_n = 14.007
  m_p = 30.973761998
  m_tot = n_ag*m_ag + (10*m_c + 6*m_h + 8*m_o)*n_k1 + (6*m_c + 12*m_h + 3*m_n + m_o + m_p)*n_k2  + (2*m_h+m_o)*n_aqua + (3*m_h+m_o+m_n)*n_k3
  
  perc_n = (3*m_n*n_k2 + m_n*n_k3) / m_tot
  perc_c = (10*m_c*n_k1 + 6*m_c*n_k2) / m_tot
  perc_h = (6*m_h*n_k1 + 12*m_h*n_k2 + 2*m_h*n_aqua + 3*m_h*n_k3 ) /m_tot
  #print(perc_n, perc_c, perc_h)
  return perc_n, perc_c, perc_h




error = 0.004
for i in np.arange(4,11):
  for j in np.arange(1,3):
    for k in np.arange(1,11):
      for l in np.arange(1,11):
        for m in np.arange(1,11):
          p1, p2, p3 = chem(i,j,k,l,m)
          if abs(p1 - 0.1485) < error and abs(p2-0.32865) <error and abs(p3 - 0.056625) < error:
            print(p1,p2,p3)
            print(i,j,k, l,m)
#chem(6, 1, 4, 5, 1)

0.14998146524231898 0.32817489296703006 0.0528497992602826
4 2 9 8 2
0.14561698478603885 0.330004423083403 0.052770173111582296
4 2 9 9 1
0.14899043451820934 0.32600641567361077 0.05324002912629184
4 2 9 9 2
0.14464946474095816 0.32781178123762095 0.05316309305964536
4 2 9 10 1


In [ ]:
|